In [42]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [43]:
df_correlaid_rdy = pd.read_csv("Correlaid_Projekte.csv", sep=";")
df_correlaid_rdy.head()

,Quelle,Projektname,Art,Einsatzbereich,Webseite-Link,Organisation,Ansprechperson,Email,Status,Kurzzusammenfassung,Unnamed: 10


***

In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()
driver.get("https://correlaid.org/daten-nutzen/projektdatenbank/")

all_projects = []

while True:
    # Parse current page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract project cards from current page
    project_cards = soup.find_all('div', class_='relative overflow-hidden rounded-lg border border-neutral-25 shadow-sm')
    
    for card in project_cards:
        all_projects.append(card)
    
    # Check current pagination status
    pagination_text = soup.find('p', class_='m-0 mx-2').get_text(strip=True)
    print(f"Current page: {pagination_text}")
    
    # Try to click next button
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Nächstes']"))
        )
        
        # Check if we're on the last page by parsing pagination text
        current_range = pagination_text.split(' of ')[0]
        total = int(pagination_text.split(' of ')[1])
        end_of_range = int(current_range.split(' - ')[1])
        
        if end_of_range >= total:
            print("Reached last page")
            break
            
        next_button.click()
        
        # Wait for new content to load
        time.sleep(2)
        
    except Exception as e:
        print(f"No more pages or error: {e}")
        break

driver.quit()

print(f"Scraped {len(all_projects)} projects total")

Current page: 1 - 8 of 40
Current page: 9 - 16 of 40
Current page: 17 - 24 of 40
Current page: 25 - 32 of 40
Current page: 33 - 40 of 40
No more pages or error: Message: 

Scraped 40 projects total


In [ ]:
# If all_projects is a list, use enumerate() to get (index, item) pairs
projectsoup_dict = {}
for index, p in enumerate(all_projects):  # <- enumerate() generates the index
    title = p.find('a', class_="mt-2 text-xl font-semibold text-base-content transition hover:text-primary").text
    orga = p.find('h4', class_="text-md ml-2 line-clamp-3 pr-12 font-semibold text-primary").text
    art = [span.text for span in p.find_all('span', class_="mr-2 line-clamp-1 inline-block whitespace-nowrap rounded bg-primary px-3 py-1 text-xs font-bold text-white")]
    bereich = [span.text for span in p.find_all('span', class_="mr-2 line-clamp-1 inline-block whitespace-nowrap rounded bg-secondary px-3 py-1 text-xs font-bold text-white")]
    teaser = p.find('span', class_="my-3 line-clamp-3 !px-0 leading-normal text-base-content").text
    projectsoup_dict[index] = {
        "Projektname": title,
        "Organisation": orga,
        "Art": art,
        "Einsatzbereich": bereich,
        "Kurzzusammenfassung": teaser,
        "soup": str(p)
    }

In [50]:
import json

with open('projectsoup_dict.json', 'w', encoding='utf-8') as f:
    json.dump(projectsoup_dict, f, ensure_ascii=False, indent=2)